In [1]:
import numpy as np, matplotlib.pyplot as plt
import cvxpy

# import cvxopt  # for GLPK_MI solver. pip install cvxopt
import scip    # for SCIP solver.    pip install pyscipopt

## Optimal solution

In [3]:
# Create two scalar optimization variables.
x = cvxpy.Variable()
y = cvxpy.Variable()

# Form objective.
objective = cvxpy.Minimize((x - y)**2)

# Create constraints.
constraints = [x + y == 5,
               1 <= x, x <= 3, 
               y >= 1, 3 >= y,
               x >= y] 
# Note: constraints can only use ==, >=, <=.  The other operators are not allowed.

# Form and solve problem.
prob = cvxpy.Problem(objective, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", np.round(prob.value,3))
print(f"optimal var: x = {np.round(x.value,3)} , y = {np.round(y.value,3)} ")

status: optimal
optimal value 0.0
optimal var: x = 2.5 , y = 2.5 


In [4]:
x = cvxpy.Variable()
y = cvxpy.Variable()

# An infeasible problem.
constraints = [x + y == 4,
               1 <= x, x <= 3, 
               y >= 5, 7 >= y] 
prob = cvxpy.Problem(cvxpy.Minimize(x), constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)  # value is irrelevant for infeasible programs


status: infeasible
optimal value inf


## Unbounded problem

In [5]:

prob = cvxpy.Problem(cvxpy.Minimize(x), constraints=[x <= 1])
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)

status: unbounded
optimal value -inf


## Integer programs

In [14]:
# Create two scalar optimization variables.
x = cvxpy.Variable(integer=True)
y = cvxpy.Variable(integer=True)

# Form objective.
objective = cvxpy.Minimize((x - y)**2)
# objective = cvxpy.Minimize((x - y))

# Create constraints.
constraints = [x + y == 5,
               1 <= x, x <= 3, 
               y >= 1, 3 >= y,
               x >= y]
# Note: constraints can only use ==, >=, <=.  The other operators are not allowed.

# Form and solve problem.
prob = cvxpy.Problem(objective, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print(f"optimal var: x = {x.value.round(3)} , y = {y.value.round(3)} ")

status: optimal
optimal value 1.0
optimal var: x = 3.0 , y = 2.0 


## Different solvers

cvxpy supports [many different solvers](https://www.cvxpy.org/tutorial/solvers/index.html).

In [15]:
prob.solve(verbose=True)

                                     CVXPY                                     
                                     v1.6.4                                    
(CVXPY) Mar 26 04:54:18 PM: Your problem has 2 variables, 6 constraints, and 0 parameters.
(CVXPY) Mar 26 04:54:18 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 26 04:54:18 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 26 04:54:18 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 26 04:54:18 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 26 04:54:18 PM: Using cached ASA map, for faster compilation 

1.0

In [ ]:
prob.solve(solver=cvxpy.SCIPY, verbose=True)  # Error - can only solve linear programs

                                     CVXPY                                     
                                     v1.6.4                                    
(CVXPY) Mar 26 04:56:03 PM: Your problem has 2 variables, 6 constraints, and 0 parameters.
(CVXPY) Mar 26 04:56:03 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 26 04:56:03 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 26 04:56:03 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 26 04:56:03 PM: Your problem is compiled with the CPP canonicalization backend.


SolverError: Either candidate conic solvers (['SCIPY']) do not support the cones output by the problem (SOC, NonNeg, Zero), or there are not enough constraints in the problem.

In [19]:
objective = cvxpy.Minimize(x - y)  # Now the objective is linear
prob_lin = cvxpy.Problem(objective, constraints)
prob_lin.solve(solver=cvxpy.SCIPY, verbose=True)  

                                     CVXPY                                     
                                     v1.6.4                                    
(CVXPY) Mar 26 04:58:04 PM: Your problem has 2 variables, 6 constraints, and 0 parameters.
(CVXPY) Mar 26 04:58:04 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 26 04:58:04 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 26 04:58:04 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 26 04:58:04 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 26 04:58:04 PM: Compiling problem (target solver=SCIPY).
(CVX

1.0

In [ ]:
# prob_lin.solve(solvers=[cvxpy.SCIPY,cvxpy.GUROBI], verbose=True)  

                                     CVXPY                                     
                                     v1.6.4                                    
(CVXPY) Mar 26 04:58:41 PM: Your problem has 2 variables, 6 constraints, and 0 parameters.
(CVXPY) Mar 26 04:58:41 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 26 04:58:41 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 26 04:58:41 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 26 04:58:41 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 26 04:58:41 PM: Compiling problem (target solver=GUROBI).
(CV

GurobiError: Unknown parameter 'solvers'

## Convex and concave

In [11]:
# We can only maximize a concave function, or minimize a convex function, but not vice versa!

prob = cvxpy.Problem(cvxpy.Maximize(x**2), constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP, even though each sub-expression is.
You are trying to maximize a function that is convex.

In [12]:
# A scalar variable.
a = cvxpy.Variable()

# Vector variable with shape (5,).
x = cvxpy.Variable(5)

# Matrix variable with shape (4, 7).
A = cvxpy.Variable((4, 7))

In [26]:
import math 
from cvxpy import log


print("\nPROBLEM: ")
print("Three cakes have to be divided among 2 people with values:")
print("2 3 4")
print("8 7 6")

# Define x,y,z = the fraction of each region given to player 1.
x = cvxpy.Variable()
y = cvxpy.Variable()
z = cvxpy.Variable()

print("\nMaximize the sum of logs:")
prob = cvxpy.Problem(
    objective   =  cvxpy.Maximize(log(2*x + 3*y + 4*z) + log(8*(1-x)+7*(1-y)+6*(1-z))),
    constraints = [0 <= x, x <= 1, 0<= y , y <= 1, 0 <= z, z <= 1])
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal product", math.exp(prob.value))
print("optimal x", round(float(x.value),5))
print("optimal y", round(float(y.value),5))
print("optimal z", round(float(z.value),5))


PROBLEM: 
Three cakes have to be divided among 2 people with values:
2 3 4
8 7 6

Maximize the sum of logs:
status: optimal
optimal value 4.150102080593772
optimal product 63.44047600905568
optimal x 0.0
optimal y 0.40476
optimal z 1.0
